# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 9 2023 9:07AM,258991,10,DNMD-36121,Emerginnova,Released
1,Mar 9 2023 9:03AM,258990,10,9247583,ISDIN Corporation,Released
2,Mar 9 2023 9:01AM,258989,10,MSP219340,Methapharm-G,Released
3,Mar 9 2023 8:56AM,258988,19,TR-RN-03092023,"GCH Granules USA, Inc.",Released
4,Mar 9 2023 8:48AM,258987,10,37818,"Digital Brands, LLC",Released
5,Mar 9 2023 8:48AM,258987,10,37819,"Digital Brands, LLC",Released
6,Mar 9 2023 8:48AM,258987,10,37820,"Digital Brands, LLC",Released
7,Mar 9 2023 8:48AM,258987,10,37821,"Digital Brands, LLC",Released
8,Mar 9 2023 8:43AM,258986,12,HH-40486,Hush Hush,Released
9,Mar 9 2023 8:43AM,258986,12,HH-40487,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
10,258987,Released,4
11,258988,Released,1
12,258989,Released,1
13,258990,Released,1
14,258991,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258987,NaN,4.0
258988,NaN,1.0
258989,NaN,1.0
258990,NaN,1.0
258991,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258960,0.0,1.0
258964,0.0,7.0
258975,0.0,1.0
258979,0.0,1.0
258980,0.0,57.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258960,0,1
258964,0,7
258975,0,1
258979,0,1
258980,0,57


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258960,0,1
1,258964,0,7
2,258975,0,1
3,258979,0,1
4,258980,0,57


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258960,,1
1,258964,,7
2,258975,,1
3,258979,,1
4,258980,,57


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 9 2023 9:07AM,258991,10,Emerginnova
1,Mar 9 2023 9:03AM,258990,10,ISDIN Corporation
2,Mar 9 2023 9:01AM,258989,10,Methapharm-G
3,Mar 9 2023 8:56AM,258988,19,"GCH Granules USA, Inc."
4,Mar 9 2023 8:48AM,258987,10,"Digital Brands, LLC"
8,Mar 9 2023 8:43AM,258986,12,Hush Hush
31,Mar 9 2023 8:34AM,258985,12,HealthCaps LLC
32,Mar 9 2023 8:30AM,258964,10,Methapharm-G
33,Mar 9 2023 8:30AM,258964,10,"Methapharm, Inc."
39,Mar 9 2023 8:30AM,258979,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 9 2023 9:07AM,258991,10,Emerginnova,,1
1,Mar 9 2023 9:03AM,258990,10,ISDIN Corporation,,1
2,Mar 9 2023 9:01AM,258989,10,Methapharm-G,,1
3,Mar 9 2023 8:56AM,258988,19,"GCH Granules USA, Inc.",,1
4,Mar 9 2023 8:48AM,258987,10,"Digital Brands, LLC",,4
5,Mar 9 2023 8:43AM,258986,12,Hush Hush,,23
6,Mar 9 2023 8:34AM,258985,12,HealthCaps LLC,,1
7,Mar 9 2023 8:30AM,258964,10,Methapharm-G,,7
8,Mar 9 2023 8:30AM,258964,10,"Methapharm, Inc.",,7
9,Mar 9 2023 8:30AM,258979,10,ISDIN Corporation,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 9 2023 9:07AM,258991,10,Emerginnova,1,
1,Mar 9 2023 9:03AM,258990,10,ISDIN Corporation,1,
2,Mar 9 2023 9:01AM,258989,10,Methapharm-G,1,
3,Mar 9 2023 8:56AM,258988,19,"GCH Granules USA, Inc.",1,
4,Mar 9 2023 8:48AM,258987,10,"Digital Brands, LLC",4,
5,Mar 9 2023 8:43AM,258986,12,Hush Hush,23,
6,Mar 9 2023 8:34AM,258985,12,HealthCaps LLC,1,
7,Mar 9 2023 8:30AM,258964,10,Methapharm-G,7,
8,Mar 9 2023 8:30AM,258964,10,"Methapharm, Inc.",7,
9,Mar 9 2023 8:30AM,258979,10,ISDIN Corporation,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 9 2023 9:07AM,258991,10,Emerginnova,1,
1,Mar 9 2023 9:03AM,258990,10,ISDIN Corporation,1,
2,Mar 9 2023 9:01AM,258989,10,Methapharm-G,1,
3,Mar 9 2023 8:56AM,258988,19,"GCH Granules USA, Inc.",1,
4,Mar 9 2023 8:48AM,258987,10,"Digital Brands, LLC",4,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 9 2023 9:07AM,258991,10,Emerginnova,1,NaN
1,Mar 9 2023 9:03AM,258990,10,ISDIN Corporation,1,NaN
2,Mar 9 2023 9:01AM,258989,10,Methapharm-G,1,NaN
3,Mar 9 2023 8:56AM,258988,19,"GCH Granules USA, Inc.",1,NaN
4,Mar 9 2023 8:48AM,258987,10,"Digital Brands, LLC",4,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 9 2023 9:07AM,258991,10,Emerginnova,1,0.0
1,Mar 9 2023 9:03AM,258990,10,ISDIN Corporation,1,0.0
2,Mar 9 2023 9:01AM,258989,10,Methapharm-G,1,0.0
3,Mar 9 2023 8:56AM,258988,19,"GCH Granules USA, Inc.",1,0.0
4,Mar 9 2023 8:48AM,258987,10,"Digital Brands, LLC",4,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1812864,22,0.0
12,517971,24,0.0
19,1553868,107,10.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1812864,22,0.0
1,12,517971,24,0.0
2,19,1553868,107,10.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,22,0.0
1,12,24,0.0
2,19,107,10.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,22.0
1,12,Released,24.0
2,19,Released,107.0
3,10,Executing,0.0
4,12,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19
Status,,,
Executing,0.0,0.0,10.0
Released,22.0,24.0,107.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19
0,Executing,0.0,0.0,10.0
1,Released,22.0,24.0,107.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19
0,Executing,0.0,0.0,10.0
1,Released,22.0,24.0,107.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()